SAM NOTES, DELETE LATER
 - need to make sure that .ipynb that student's will work out of are running in same virutal enviroment as everything else
 - embed .ipynb into quarto notebook (get book to build from those examples)
 - use Ryan Abernathey's [post](https://medium.com/pangeo/closed-platforms-vs-open-architectures-for-cloud-native-earth-system-analytics-1ad88708ebb6) to help frame introduction

## Learning Objectives

- Understand what Google Earth Engine provides and its applications
- Learn about some real-world applications of Google Earth Engine
- Learn how to get started using Google Earth Engine on your own computer
- Learn how to find and access Google Earth Engine Data

## Introduction (15-20min)

SAM NOTES, DELETE LATER 
- have Ingmar help frame utility of GEE in intro

[Google Earth Engine](https://earthengine.google.com/) is a geospatial processing platform powered by Google Cloud Platform. It contains over 30 years of satellite imagery and geospatial datasets that are continually updated and available instantly. The Earth Engine API is available in Python (and JavaScript) for anyone with an account to access and analyze data.

ADD SOME COOL IMAGERY HERE

Explore the public [Earth Engine Data Catalog](https://developers.google.com/earth-engine/datasets/catalog) which includes a variety of standard Earth science raster datasets. Browse by [dataset tags](https://developers.google.com/earth-engine/datasets/tags) or by satellites ([Landsat](https://developers.google.com/earth-engine/datasets/catalog/landsat), [MODIS](https://developers.google.com/earth-engine/datasets/catalog/modis), [Sentinel](https://developers.google.com/earth-engine/datasets/catalog/sentinel)).

SAM NOTES, DELETE LATER
- typical: download data and work locally
- new way:"Moving compute to data" -- GEE is a great example of this

## Getting started with Google Earth Engine (GEE) on your own machine (40 min)

SAM NOTES, DELETE LATER

- remove `#| eval: false` once code actually runs to embed outputs
- don't really have time to actually do this in class? include instructions for those who want to try it out on their own later?
- Content borrowed from Dr. Samantha Stevenson's [guide to installling Jupyter/Google Earth Engine onyour personal laptop](https://docs.google.com/document/d/1P1c6Ef7D1OoGaTaOrPDO0m5LBN1okpGlU0OXKKzsoi0/edit).
- PREREQUISITE: need conda (see Earth Engine API installion instructions [here](https://developers.google.com/earth-engine/guides/python_install-conda)) -- haven't added these instructions below yet
- also, not working on my machine at the moment. issue with library installs? starts with `earthengine authenticate`


1. Install the Google Earth Engine API

SAM NOTES, DELETE LATER 
- MOVE MOST OF THIS TO COURSE SETUP (not using conda) BUT KEEP GEE ACTIVATION STEP HERE

- Create an environment where the Google Earth Engine API will live. This ensures that it and it's dependent packages will not cause versioning issues with your base environment (or other environments). We'll call our environment `gee_env`.


```{bash}
#| eval: false
conda create --name gee_env
```


- Activate your environment so your machine knows where to store subsequent installs.


```{bash}
#| eval: false
conda activate gee_env
```


You'll know your environment is activated successfully when `(gee_env)` appears before the prompt in your terminal window (as opposed to `(base)`, for example).

- Install the Google Earth Engine API in your `gee_env`


```{bash}
#| eval: false
conda install -c conda-forge earthengine-api
```


2. Sign up for a GEE Account

GEE is currently free for educational use. Sign up for an account at https://signup.earthengine.google.com (you'll need this to authenticate in the next step).

3. Set up GEE Authentication

In order to begin using GEE, you'll need to connect your GEE envionment (`gee_env`) to the authentication credentials associated with your Google account. This will need to be done each time you connect to GEE, but should only be done once per session. 

- On the command line, type:


```{bash}
#| eval: false
earthengine authenticate
```


This should launch a browser window where you can login with your Google account to the Google Earth Engine Authenticator. Following the prompts will generate a code, which you'll then need to copy and paste back onto the command line. This will be saved as an authentication token so you won't need to go through this process again until the next time you start a new session.

4. Install necessary packages (if you don't already have them)


```{bash}
#| eval: false
pip install ee # Earth Engine API package
pip install geemap # package for interactive maping with GEE 
pip install pandas # contains useful tools for data manipulation (may not need this)
```


## Visualize global precipitation data using Google Earth Engine 

*Content for this section was adapted from Dr. Sam Stevenson's [Visualizing global precipitation using Google Earth Engine](https://github.com/samanthastevenson/EDS220_Fall2021/blob/main/Week1_ERA5Maps_GEE.ipynb) lesson, given in her [EDS 220 course](https://samanthastevenson.github.io/EDS220_site/) in Fall 2021.*

1. Import necessary packages


In [ ]:
#| eval: false
import ee # MODULENOTFOUNDERROR
import geemap
import pandas as pd

2. Create an interactive basemap

The default basemap is (you guessed it) Google Maps. The following code displays an empty Google Map that you can manipulate just like you would in the typical Google Maps interface. Do this using the `Map` method from the `geemap` library. We'll also center the map at a specified latitude and longitude (here, 40N, 100E), set a zoom level, and save our map as an object called `myMap`.


In [ ]:
#| eval: false
myMap = geemap.Map(center = [40, -100], zoom = 2)
myMap

3. Load ERA5 Image Collections from GEE

* NOTE: ADC has worked with these data -- took 3 weeks to download
* EE colleciton is all you need to load and analyze imgage collection
* precursor to Ingmar's stuff

We'll be using the ERA5 daily aggregates reanalysis dataset, produced by the European Centre for Medium-Range Weather Forecasts (ECMWF), found [here](https://developers.google.com/earth-engine/datasets/catalog/ECMWF_ERA5_DAILY), which models atmospheric weather observations. We'll load the `total_precipitation` field (check out the dataset metadata on [here](https://developers.google.com/earth-engine/datasets/catalog/ECMWF_ERA5_DAILY#bands)).

The `ImageCollection` method extracts a set of individual images that satisfies some criterion that you pass to GEE through the `ee` package. This is stored as an **ImageCollection** object which can be filtered and processed in various ways. We can pass the `ImageCollction` method agruments to tell GEE which data we want to retrieve. Below, we retrieve all daily ERA5 data (so we can see individual rain events).


In [ ]:
#| eval: false
weatherData = ee.ImageCollection('ECMWF/ERA5/DAILY')

4. Select an image to plot

To plot a map over our Google Maps basemap, we need an "Image" rather than an "ImageCollection." ERA5 contains many different climate variables, so we need to pick what we'd like to plot. We'll use the `.select` method to choose the parameter(s) we're interested in from our `weatherData` object.


In [ ]:
#| eval: false
precip = weatherData.select("total_precipitation")

We can look at our `precip` object using the `print` method to see that it's still an "ImageCollection" which contains daily infomration from 1979 to 2020. 


In [ ]:
#| eval: false
print(precip)

We want to filter it down to a single field for a time of interest -- let's say December 1-2, 2019. We apply the `.filter` method to our `precip` object and apply the `ee.Filter.date` method (from the `ee` package) to filter for data from our chosen date range. We also apply the `.mean` method, which takes whatever precedes it and calculates the average.


In [ ]:
#| eval: false
precip_filtered = precip.filter(ee.Filter.date('2019-12-01', '2019-12-02')).mean()

5. Add data to map

We can fist use the `setCenter` method to tell the map where to center itself. It takes the longitude and latitude as the first two coordinates, followed by the zoom level.


In [ ]:
#| eval: false
Map.setCenter(-152.505706, 59.432367, 2) # Cook Inlet, Alaska (WE CAN CHANGE THIS LOCATION)

Next, set a color palette to use when plotting the data layer. The following is a palette specified for precipitation in the GEE description page for ERA5. GEE has lots of color tables like this that you can look up. 


In [ ]:
#| eval: false
precip_palette = {
    'min':0,
    'max':0.1,
    'palette': ['#FFFFFF', '#00FFFF', '#0080FF', '#DA00FF', '#FFA400', '#FF0000']
}

Finally, plot our filtered data, `precip_filtered` on top of our basemap using the `.addLayer` method. We'll aslo pass it our visualization parameters (colors and ranges stored in `precip_palette`, the name of the data field `total precipitation`, and opacity so that we can see the basemap underneath)


In [ ]:
#| eval: false
Map.addLayer(precip_filtered, precip_palette, 'total precipitation', opacity = 0.3)

## INGMAR'S DEMONSTRATION HERE?(30-40 min)
* optical remote sensing
1. load and visualize image
  * site? - somewhere arctic, lake drainage Alaska
2. calculate and visualize indices
3. 

## Conclusion/Summary 

* lessons learned
* utilities
* etc.

## Other Resources

- [GEE Code Editor](https://code.earthengine.google.com/) is a web-based IDE for using GEE (JavaScript)
- [GEE Datasets](https://developers.google.com/earth-engine/datasets) is the main resource for "official" GEE Datasets
- [GEE Community Datasets](https://samapriya.github.io/awesome-gee-community-datasets/) is the main resource for "community" GEE Datasets